## Homework 6

*Jackson Rudoff*

**October 26th, 2022**

In [1]:
import numpy as np  
import pytest 
import pandas as pd 


### A

The first question is straightforward, just need to read in the function, get the output for the word in question, and make a function that asserts that it will provide the correct output. 

**Note**: I am getting the expected output in advance from using the cipher function, but excluding it for brevity. 

In [2]:
def cipher(text, shift, encrypt=True):
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    new_text = ''
    for c in text:
        index = alphabet.find(c)
        if index == -1:
            new_text += c
        else:
            new_index = index + shift if encrypt == True else index - shift
            new_index %= len(alphabet)
            new_text += alphabet[new_index:new_index+1]
    return new_text

In [3]:
def test_word():
    example = 'word'
    shift = 4
    expected = 'Asvh'
    actual = cipher(example, shift)
    assert actual == expected


test_word()

No output is good here, since it is not intended to generate any outputs or errors. If it didn't work, however, because the function was not programmed correctly, the ouput would look like this. 

In [4]:
def cipher_broken(text, shift, encrypt=True):
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    new_text = ''
    for c in text:
        index = alphabet.find(c)
        if index == -1:
            new_text += c
        else:
            new_index = index * shift if encrypt == True else index - shift #add incorrect math
            new_index %= len(alphabet)
            new_text += alphabet[new_index:new_index+1]
    return new_text

def test_word():
    example = 'word'
    shift = 4
    expected = 'Asvh'
    actual = cipher_broken(example, shift)
    assert actual == expected, "Unexpected output."

test_word()

AssertionError: Unexpected output.

## B

Again, not much to this one. It just needs to feed a negative shift into the function. 

In [5]:
def test_neg():
    test_text = "The New York Mets were once good at baseball and then they sucked"
    example_shift = -2
    expected = "Rfc Lcu Wmpi Kcrq ucpc mlac emmb Yr ZYqcZYjj Ylb rfcl rfcw qsaicb"
    actual = cipher(test_text, example_shift)
    assert actual == expected

test_neg()

## C

The code indicates that any characters not in the alphabet will be added (but not shifted), so the output should include shifted alphabetic characters and retained special characters. 

In [6]:
def test_special():
    test_text = "UhOh!?!%"
    test_shift = 5
    expected = "ZmTm!?!%"
    actual = cipher(test_text, test_shift)
    assert expected == actual

test_special()


No output, so it does do what we'd expect with special characters. 

## D

Here we need to add an exception to make sure it only accepts a **str** input for shift. 

In [7]:
def cipher(text, shift, encrypt=True):
    assert type(shift) != str, "Cannot pass string to shift."
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    new_text = ''
    for c in text:
        index = alphabet.find(c)
        if index == -1:
            new_text += c
        else:
            new_index = index + shift if encrypt == True else index - shift
            new_index %= len(alphabet)
            new_text += alphabet[new_index:new_index+1]
    return new_text



Check that it catches:

In [8]:
cipher("two", "two")

AssertionError: Cannot pass string to shift.

For this test I'm gonna use the pytest method **.raises()** since it makes test a lot cleaner. If it raises an assertion, then it will pass. 

In [10]:
def test_assert():
    with pytest.raises(AssertionError):
        test_word = "two"
        test_shift = "two"
        cipher(test_word,test_shift)


In [11]:
test_assert()

What if it doesn't raise an assertion?

In [12]:
def cipher(text, shift, encrypt=True):
    #assert type(shift) != str, "Cannot pass string to shift." 
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    new_text = ''
    for c in text:
        index = alphabet.find(c)
        if index == -1:
            new_text += c
        else:
            new_index = index + shift if encrypt == True else index - shift
            new_index %= len(alphabet)
            new_text += alphabet[new_index:new_index+1]
    return new_text

def test_assert():
    with pytest.raises(AssertionError):
        test_word = "two"
        test_shift = "two"
        cipher(test_word,test_shift)

test_assert()

TypeError: unsupported operand type(s) for +: 'int' and 'str'

## E

For this question, I have put all the questions into a separate .py file (*test_cipher.py*), along with the cipher function itself (*cipher.py*). Both of these are located in the /tests folder. 

In [13]:
!pytest -vv tests/test_cipher.py

============================= test session starts ==============================
platform darwin -- Python 3.9.1, pytest-7.1.3, pluggy-1.0.0 -- /Users/jacksonrudoff/.pyenv/versions/3.9.1/bin/python3
cachedir: .pytest_cache
rootdir: /Users/jacksonrudoff/Documents/QMSS/Fall_2022/mds_course/Rudoff_Jackson/hw06
plugins: Faker-15.0.0
collected 4 items                                                              

tests/test_cipher.py::test_word PASSED                                   [ 25%]
tests/test_cipher.py::test_neg PASSED                                    [ 50%]
tests/test_cipher.py::test_special PASSED                                [ 75%]
tests/test_cipher.py::test_assert PASSED                                 [100%]

============================== 4 passed in 0.25s ===============================


## F

Here my approach is to parametrize the examples so that the test function can be fed easily. I also created a small csv with the examples so that I can use a fixture to provide the example parameters. I also added this to a new testing suite called *parametrize.py* to demonstrate how we've condensed four tests into one. 

In [25]:
@pytest.fixture
def words_csv():
    df = pd.read_csv('data/words.csv')
    return df

@pytest.mark.parametrize("example, shift, expected", [
    (0, 4, 'TCxlsr'),
    (1, 5, 'YJXYNSL'),
    (2, 7, 'kpmmpjBsA'), #intentionally false example
    (3, 8,'epG qA qB Aw lqnnqkCtB.')
])
def test_parametrize(words_csv, example, shift, expected):
    result = cipher(words_csv.loc[example]['Extract'], shift)
    assert result == expected

In [27]:
!pytest -vv tests/parametrize.py::test_parametrize

============================= test session starts ==============================
platform darwin -- Python 3.9.1, pytest-7.1.3, pluggy-1.0.0 -- /Users/jacksonrudoff/.pyenv/versions/3.9.1/bin/python3
cachedir: .pytest_cache
rootdir: /Users/jacksonrudoff/Documents/QMSS/Fall_2022/mds_course/Rudoff_Jackson/hw06
plugins: Faker-15.0.0
collected 4 items                                                              

tests/parametrize.py::test_parametrize[0-4-TCxlsr] PASSED                [ 25%]
tests/parametrize.py::test_parametrize[1-5-YJXYNSL] PASSED               [ 50%]
tests/parametrize.py::test_parametrize[2-7-kpmmpjBsA] PASSED             [ 75%]
tests/parametrize.py::test_parametrize[3-8-epG qA qB Aw lqnnqkCtB.] PASSED [100%]

============================== 4 passed in 0.24s ===============================


What if the parametrize feeds the wrong shift (to show what a non-passing test looks like)?

In [20]:
@pytest.fixture
def words_csv():
    df = pd.read_csv('data/words.csv')
    return df

@pytest.mark.parametrize("example, shift, expected", [
    (0, 4, 'TCxlsr'),
    (1, 5, 'YJXYNSL'),
    (2, 7, 'kpmmpjBsA'), #intentionally false example
    (3, 8,'epG qA qB Aw lqnnqkCtB.')
])
def test_parametrize(words_csv, example, shift, expected):
    result = cipher(words_csv.loc[example]['Extract'], shift+1)
    assert result == expected

In [28]:
!pytest -vv tests/parametrize_bad.py

============================= test session starts ==============================
platform darwin -- Python 3.9.1, pytest-7.1.3, pluggy-1.0.0 -- /Users/jacksonrudoff/.pyenv/versions/3.9.1/bin/python3
cachedir: .pytest_cache
rootdir: /Users/jacksonrudoff/Documents/QMSS/Fall_2022/mds_course/Rudoff_Jackson/hw06
plugins: Faker-15.0.0
collected 4 items                                                              

tests/parametrize_bad.py::test_parametrize[0-4-TCxlsr] FAILED            [ 25%]
tests/parametrize_bad.py::test_parametrize[1-5-YJXYNSL] FAILED           [ 50%]
tests/parametrize_bad.py::test_parametrize[2-7-kpmmpjBsA] FAILED         [ 75%]
tests/parametrize_bad.py::test_parametrize[3-8-epG qA qB Aw lqnnqkCtB.] FAILED [100%]

=================================== FAILURES ===================================
_________________________ test_parametrize[0-4-TCxlsr] _________________________

words_csv =    Example                  Extract
0        1                   Python
1        2    

## G

Lastly, we just need to do an integration test. This can be easily done by just setting up another parametrize. The test itself is also fairly straightforward. Simply feed the string example into the cipher to be encrypted, and then feed that encryption output to the decryption setting. If all the internal functionality is working as expected, then the final output should equal the string from the parametrization. 

In [18]:
@pytest.mark.parametrize("example, shift", [
    ('It', 1),
    ('was', 2),
    ('the', 3),
    ('best', 4),
    ('of', 5),
    ('times', 6),
    ('it', 7),
    ('was', 8),
    ('the', 9),
    ('worst', 10),
    ('of', 11),
    ('times', 12)
])

def test_integration(example, shift):
    encrypt = cipher(example, shift, encrypt=True)
    decrypt = cipher(encrypt, shift, encrypt=False)
    assert decrypt == example


In [29]:
!pytest -vv tests/parametrize.py::test_integration

============================= test session starts ==============================
platform darwin -- Python 3.9.1, pytest-7.1.3, pluggy-1.0.0 -- /Users/jacksonrudoff/.pyenv/versions/3.9.1/bin/python3
cachedir: .pytest_cache
rootdir: /Users/jacksonrudoff/Documents/QMSS/Fall_2022/mds_course/Rudoff_Jackson/hw06
plugins: Faker-15.0.0
collected 12 items                                                             

tests/parametrize.py::test_integration[It-1] PASSED                      [  8%]
tests/parametrize.py::test_integration[was-2] PASSED                     [ 16%]
tests/parametrize.py::test_integration[the-3] PASSED                     [ 25%]
tests/parametrize.py::test_integration[best-4] PASSED                    [ 33%]
tests/parametrize.py::test_integration[of-5] PASSED                      [ 41%]
tests/parametrize.py::test_integration[times-6] PASSED                   [ 50%]
tests/parametrize.py::test_integration[it-7] PASSED                      [ 58%]
tests/parametrize.py::test_i